# Install Java, Spark, and Findspark
# Set Environment Variables

In [15]:
!apt-get update

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,192 kB]
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,531 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu 

In [16]:
# Install Java
!apt-get install openjdk-11-jdk-headless -qq > /dev/null

In [17]:
#Download the version in Spark
!wget https://archive.apache.org/dist/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz

--2024-12-08 12:18:42--  https://archive.apache.org/dist/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
Resolving archive.apache.org (archive.apache.org)... 65.108.204.189, 2a01:4f9:1a:a084::2
Connecting to archive.apache.org (archive.apache.org)|65.108.204.189|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 400395283 (382M) [application/x-gzip]
Saving to: ‘spark-3.5.0-bin-hadoop3.tgz.1’

spark-3.5.0-bin-had 100%[===================>] 381.85M  26.9MB/s    in 15s     

2024-12-08 12:18:58 (25.3 MB/s) - ‘spark-3.5.0-bin-hadoop3.tgz.1’ saved [400395283/400395283]



In [18]:
!tar xf spark-3.5.0-bin-hadoop3.tgz


In [19]:
!pip install -q findspark

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

In [22]:
#Set the locations where Spark and Java are installed.
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"


In [23]:
#This will start a local Spark session.
import findspark
findspark.init()

In [24]:
#Create Spark Session
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master('local[*]')\
        .appName('SparkColab')\
        .getOrCreate()

# Import DataSet


In [30]:
df = spark.read.csv('/content/netflix_titles.csv', header=True, inferSchema=True)

In [31]:
df.printSchema()

root
 |-- show_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- director: string (nullable = true)
 |-- cast: string (nullable = true)
 |-- country: string (nullable = true)
 |-- date_added: string (nullable = true)
 |-- release_year: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- listed_in: string (nullable = true)
 |-- description: string (nullable = true)



In [32]:
df.show()

+--------+-------+--------------------+--------------------+--------------------+--------------------+-----------------+------------+--------+--------+--------------------+--------------------+
| show_id|   type|               title|            director|                cast|             country|       date_added|release_year|  rating|duration|           listed_in|         description|
+--------+-------+--------------------+--------------------+--------------------+--------------------+-----------------+------------+--------+--------+--------------------+--------------------+
|81145628|  Movie|Norm of the North...|Richard Finn, Tim...|Alan Marriott, An...|United States, In...|September 9, 2019|        2019|   TV-PG|  90 min|Children & Family...|Before planning a...|
|80117401|  Movie|Jandino: Whatever...|                NULL|    Jandino Asporaat|      United Kingdom|September 9, 2016|        2016|   TV-MA|  94 min|     Stand-Up Comedy|"Jandino Asporaat...|
|70234439|TV Show|  Transforme

In [34]:
df_movie = df.select('title','director','country','release_year','duration')

In [35]:
df_movie.show()

+--------------------+--------------------+--------------------+------------+--------+
|               title|            director|             country|release_year|duration|
+--------------------+--------------------+--------------------+------------+--------+
|Norm of the North...|Richard Finn, Tim...|United States, In...|        2019|  90 min|
|Jandino: Whatever...|                NULL|      United Kingdom|        2016|  94 min|
|  Transformers Prime|                NULL|       United States|        2013|1 Season|
|Transformers: Rob...|                NULL|       United States|        2016|1 Season|
|        #realityhigh|    Fernando Lebrija|       United States|        2017|  99 min|
|             Apaches|                NULL|               Spain|        2016|1 Season|
|            Automata|         Gabe Ibáñez|Bulgaria, United ...|        2014| 110 min|
|Fabrizio Copano: ...|Rodrigo Toro, Fra...|               Chile|        2017|  60 min|
|        Fire Chasers|                NULL|

In [43]:
from pyspark.sql.functions import current_date

In [44]:
# Alterar o tipo da coluna "release_year" para inteiro e adicionar a data atual
df_transformed = df.withColumn("release_year", df["release_year"].cast("int")) \
                   .withColumn("data_corrente", current_date())

df_transformed.printSchema()
df_transformed.show(5)

root
 |-- show_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- director: string (nullable = true)
 |-- cast: string (nullable = true)
 |-- country: string (nullable = true)
 |-- date_added: string (nullable = true)
 |-- release_year: integer (nullable = true)
 |-- rating: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- listed_in: string (nullable = true)
 |-- description: string (nullable = true)
 |-- data_corrente: date (nullable = false)

+--------+-------+--------------------+--------------------+--------------------+--------------------+-----------------+------------+--------+--------+--------------------+--------------------+-------------+
| show_id|   type|               title|            director|                cast|             country|       date_added|release_year|  rating|duration|           listed_in|         description|data_corrente|
+--------+-------+--------------------+-------------------

In [46]:
# Filtrar filmes lançados após 2015
df_filtrado = df_transformed.filter(df_transformed["release_year"] > 2015)
df_filtrado.show(5, truncate=False)


+--------+-------+---------------------------------------+------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------+-----------------+------------+------+--------+-----------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------+-------------+
|show_id |type   |title                                  |director                |cast                                                                                                                                                                    |country                                 |date_added       |release_year|rating|duration|listed_in                                                        |description            

In [49]:
# Contar a quantidade de títulos por gênero (listed_in)
df_generos = df_transformed.groupBy("listed_in").count()
df_generos = df_generos.orderBy(col("count").desc())

df_generos.show(truncate=False)

+----------------------------------------------------+-----+
|listed_in                                           |count|
+----------------------------------------------------+-----+
|Documentaries                                       |299  |
|Stand-Up Comedy                                     |273  |
|Dramas, International Movies                        |248  |
|Dramas, Independent Movies, International Movies    |186  |
|Comedies, Dramas, International Movies              |174  |
|Kids' TV                                            |159  |
|Documentaries, International Movies                 |150  |
|Children & Family Movies, Comedies                  |129  |
|Children & Family Movies                            |120  |
|Comedies, International Movies                      |120  |
|Dramas, International Movies, Romantic Movies       |108  |
|Action & Adventure, Dramas, International Movies    |103  |
|Comedies, International Movies, Romantic Movies     |97   |
|Dramas, International M

In [52]:
from pyspark.sql.functions import regexp_replace

In [53]:
df_duracao = df_transformed.withColumn("duration_min",
                                       regexp_replace("duration", " min", "").cast("int"))

# Filtrar títulos com mais de 100 minutos
df_longos = df_duracao.filter(df_duracao["duration_min"] > 100)
df_longos.show(5, truncate=False)

+--------+-----+------------------+---------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------+-----------------+------------+------+--------+----------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------+-------------+------------+
|show_id |type |title             |director                   |cast                                                                                                                                                           |country                               |date_added       |release_year|rating|duration|listed_in                                           |description                                                                                   

In [54]:
from pyspark.sql.functions import max

In [56]:
# Consultar o título com a maior duração
df_maior_duracao = df_duracao.agg(max("duration_min").alias("maior_duracao"))
df_maior_duracao.show()

+-------------+
|maior_duracao|
+-------------+
|          312|
+-------------+



In [57]:
# Mostrar detalhes do título com a maior duração
maior_duracao = df_maior_duracao.collect()[0]["maior_duracao"]
df_titulo_maior_duracao = df_duracao.filter(df_duracao["duration_min"] == maior_duracao)
df_titulo_maior_duracao.show(truncate=False)

+--------+-----+--------------------------+--------+-------------------------------------------------------------------------+-------------+-----------------+------------+------+--------+----------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------+-------------+------------+
|show_id |type |title                     |director|cast                                                                     |country      |date_added       |release_year|rating|duration|listed_in                                     |description                                                                                                                                           |data_corrente|duration_min|
+--------+-----+--------------------------+--------+-------------------------------------------------------------------------+-------------+-----------------+------------+---

In [66]:
from pyspark.sql.functions import year, to_date

In [67]:
# Converter a coluna "date_added" para DateType
df_transformed = df_transformed.withColumn("date_added", to_date("date_added", "MMMM d, yyyy"))

# Criar uma coluna com o ano de adição
df_ano_adicionado = df_transformed.withColumn("ano_adicionado", year("date_added"))


df_ano_adicionado.show(5)

+--------+-------+--------------------+--------------------+--------------------+--------------------+----------+------------+--------+--------+--------------------+--------------------+-------------+--------------+
| show_id|   type|               title|            director|                cast|             country|date_added|release_year|  rating|duration|           listed_in|         description|data_corrente|ano_adicionado|
+--------+-------+--------------------+--------------------+--------------------+--------------------+----------+------------+--------+--------+--------------------+--------------------+-------------+--------------+
|81145628|  Movie|Norm of the North...|Richard Finn, Tim...|Alan Marriott, An...|United States, In...|2019-09-09|        2019|   TV-PG|  90 min|Children & Family...|Before planning a...|   2024-12-08|          2019|
|80117401|  Movie|Jandino: Whatever...|                NULL|    Jandino Asporaat|      United Kingdom|2016-09-09|        2016|   TV-MA| 

In [68]:
# Contar a frequência de títulos adicionados por ano
df_frequencia_ano = df_ano_adicionado.groupBy("ano_adicionado").count()
df_frequencia_ano = df_frequencia_ano.orderBy(col("ano_adicionado").asc())

df_frequencia_ano.show(truncate=False)

+--------------+-----+
|ano_adicionado|count|
+--------------+-----+
|NULL          |661  |
|2008          |2    |
|2009          |2    |
|2010          |1    |
|2011          |13   |
|2012          |7    |
|2013          |9    |
|2014          |19   |
|2015          |73   |
|2016          |412  |
|2017          |1182 |
|2018          |1627 |
|2019          |2055 |
|2020          |173  |
+--------------+-----+



In [78]:
# Usando IN: Filtrar registros onde o país está em uma lista de valores
countries_to_include = ["United States", "United Kingdom", "France"]
df_included = df_ano_adicionado.filter(col("country").isin(countries_to_include))
df_included.show(truncate=False)

+--------+-------+--------------------------------+-------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------+----------+------------+--------+--------+------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------+-------------+--------------+
|show_id |type   |title                           |director                       |cast                                                                                                                                                                                                                                                 |country       |date_added|release_year|rating  |durat

In [77]:
# Usando NOT IN: Filtrar registros onde o país NÃO está em uma lista de valores
countries_to_exclude = ["United States", "United Kingdom"]
df_excluded = df_ano_adicionado.filter(~col("country").isin(countries_to_exclude))
df_excluded.show(truncate=False)

+--------+-------+----------------------------------------+-------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------+----------+------------+------+--------+-----------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------+-------------+--------------+
|show_id |type   |title                                   |director                       |cast                                                                                                                                                                                                                                 |country         

In [73]:
# Filtrar os registros onde o ano de adição NÃO é 2017 OU o país é "United States"
df_filtered = df_ano_adicionado.filter(
    ~(col("ano_adicionado") != 2017) | (col("country") == "United States")
)
df_filtered.show()

+--------+-------+--------------------+--------------------+--------------------+--------------------+----------+------------+--------+--------+--------------------+--------------------+-------------+--------------+
| show_id|   type|               title|            director|                cast|             country|date_added|release_year|  rating|duration|           listed_in|         description|data_corrente|ano_adicionado|
+--------+-------+--------------------+--------------------+--------------------+--------------------+----------+------------+--------+--------+--------------------+--------------------+-------------+--------------+
|70234439|TV Show|  Transformers Prime|                NULL|Peter Cullen, Sum...|       United States|2018-09-08|        2013|TV-Y7-FV|1 Season|            Kids' TV|With the help of ...|   2024-12-08|          2018|
|80058654|TV Show|Transformers: Rob...|                NULL|Will Friedle, Dar...|       United States|2018-09-08|        2016|   TV-Y7|1

In [74]:
# Using SQL
# Registrar o DataFrame como uma tabela SQL temporária
df_ano_adicionado.createOrReplaceTempView("movies_table")

# Escrever uma consulta SQL para filtrar os dados
df_sql_filtered = spark.sql("""
    SELECT *
    FROM movies_table
    WHERE (ano_adicionado NOT IN (2017, 2018) OR country = 'United States')
""")


df_sql_filtered.show(truncate=False)

+--------+-------+------------------------------------------------+--------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------+----------+------------+--------+--------+------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------+-------------+--------------+
|show_id |type   |title                                           |director                  |cast                                                                                                                                                                              |country                                 |date_added|release_year|rating  |duration|listed_in                                       |descrip

In [ ]:
# Converter para Pandas
df_pandas = df_generos.toPandas()

# Criar visualização com Matplotlib ou Seaborn
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
df_pandas.head(10).plot(kind="bar", x="listed_in", y="count", legend=False, color="teal")
plt.title("Top 10 Gêneros com Mais Títulos", fontsize=16)
plt.xlabel("Gênero")
plt.ylabel("Quantidade de Títulos")
plt.xticks(rotation=45)
plt.show()
